#  ==================================
# IMPORTS
#  ==================================

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

sns.set_theme()

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# no text wrapping
display(HTML("<style>.dataframe td { white-space: nowrap; }</style>"))

# pandas formatting
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 100)

In [2]:
# import dataframes
df_SD = pd.read_pickle('df_SD.pickle')
df_FD = pd.read_pickle('df_FD.pickle')
df_LF = pd.read_pickle('df_LF.pickle')
df_Site = pd.read_pickle('df_Site.pickle')
df_TrapSupervisors = pd.read_pickle('df_TrapSupervisors.pickle')

### Helper Function to UPDATE ID to Match Updated DATETIME and SITE

In [3]:
def create_new_id(row):
    """only run before disambiguating month portion of ids for FD and LF"""
    return str(row.id)[0:4] + str(row.DATETIME.month).rjust(2,'0') + str(row.DATETIME.day).rjust(2, '0') + str(row.SITE1).rjust(2, '0')

#  ==================================
# FISH DETAILS
#  ==================================

## Combine Age 2 and Age 3 Columns

In [4]:
# check if age2 and age3 occur at the same time
(
    df_FD[df_FD['Ager_2'].notnull() & df_FD['Ager_3'].notnull()].shape[0],
    df_FD[df_FD['AGE_2'].notnull() & df_FD['AGE_3'].notnull()].shape[0],
    df_FD[df_FD['FSP_2'].notnull() & df_FD['FSP_3'].notnull()].shape[0], 
    df_FD[df_FD['Comments_2'].notnull() & df_FD['Comments_3'].notnull()].shape[0]
)

(0, 0, 0, 0)

In [5]:
# how many Ager 2 columns
df_FD[df_FD.Ager_2.notnull() | df_FD.AGE_2.notnull() | df_FD.FSP_2.notnull() | df_FD.Comments_2.notnull()].shape[0]

1683

In [6]:
# how many Ager 3 columns
df_FD[df_FD.Ager_3.notnull() | df_FD.AGE_3.notnull() | df_FD.FSP_3.notnull() | df_FD.Comments_3.notnull()].shape[0]

2339

In [7]:
# combine aging 2 and aging 3 into one set of data
df_FD['Ager_2'] = df_FD['Ager_2'].fillna(df_FD['Ager_3'])
df_FD['AGE_2'] = df_FD['AGE_2'].fillna(df_FD['AGE_3'])
df_FD['FSP_2'] = df_FD['FSP_2'].fillna(df_FD['FSP_3'])
df_FD['Comments_2'] = df_FD['Comments_2'].fillna(df_FD['Comments_3'])

In [8]:
# how many Ager 2 columns
df_FD[df_FD.Ager_2.notnull() | df_FD.AGE_2.notnull() | df_FD.FSP_2.notnull() | df_FD.Comments_2.notnull()].shape[0]

4022

## FLAG_SITE: ambiguous site data

In [9]:
# TODO: if site is changed away from SITE1, update id to match

# df_FD.loc[df_FD.FLAG_SITE==True]

## FLAG_AM_PM_PERIOD: leave as is

## FLAG_SEX: B and A? -> null

In [10]:
df_FD.loc[df_FD.FLAG_SEX==True, 'SEX'] = np.nan

In [11]:
df_FD.loc[:, 'SEX'].unique()

array(['F', 'M', 'U', nan], dtype=object)

## FLAG_MATURITY: 44=4, 0=null

In [12]:
df_FD.loc[df_FD.MATURITY==44, 'MATURITY'] = 4
df_FD.loc[df_FD.MATURITY==0, 'MATURITY'] = np.nan

## FLAG_FSP_1: 33=3

In [13]:
df_FD.loc[df_FD.FSP_1==33, 'FSP_1'] = 3

## FLAG_FL_STD: 10x off

In [14]:
# CONFIRM
df_FD.loc[df_FD.FLAG_FL_STD==True, 'FL_STD'] *= 10
df_FD.loc[df_FD.FLAG_FL_STD==True, 'FL_STD']

34923   261.000
34924   218.000
34925   260.000
34926   283.000
34927   219.000
          ...  
35504   227.000
35505   236.000
35506   208.000
35507   228.000
35508   202.000
Name: FL_STD, Length: 264, dtype: float64

## FLAG_FL_WET_FROZEN: 5 typos

In [15]:
# STD doesn't need to be converted here, it will be standardised in dm_apps from the frozen length and frozen condition
df_FD.loc[df_FD.FLAG_FL_WET_FROZEN==True, ['FL_WET', 'FL_STD']] = np.nan
df_FD.loc[df_FD.FLAG_FL_WET_FROZEN==True, 'FL_FROZEN'] = [259, 251, 228, 200, 204]
df_FD.loc[df_FD.FLAG_FL_WET_FROZEN==True, 'fish_length'] = [259, 251, 228, 200, 204]
df_FD.loc[df_FD.FLAG_FL_WET_FROZEN==True, ['CONDITION', 'FL_WET', 'FL_FROZEN', 'FL_STD', 'fish_length']]

,CONDITION,FL_WET,FL_FROZEN,FL_STD,fish_length
35249,Frozen,<NA>,259,NaN,259
35392,Frozen,<NA>,251,NaN,251
35478,Frozen,<NA>,228,NaN,228
35479,Frozen,<NA>,200,NaN,200
35507,Frozen,<NA>,204,NaN,204


## FLAG_WEIGHT_OUTLIER: 

In [16]:
df_FD.loc[df_FD.FLAG_WEIGHT_OUTLIER==True, 'WEIGHT']

724      519.000
735      539.000
956      501.000
1777     512.000
5477     503.000
35212   1934.000
Name: WEIGHT, dtype: float64

In [17]:
# CONFIRM
df_FD.loc[df_FD.WEIGHT>1000, 'WEIGHT'] = np.nan
df_FD.loc[df_FD.FLAG_WEIGHT_OUTLIER==True, 'WEIGHT']

724     519.000
735     539.000
956     501.000
1777    512.000
5477    503.000
35212       NaN
Name: WEIGHT, dtype: float64

## FLAG_GONAD_OUTLIER:

In [18]:
# CONFIRM
df_FD.loc[df_FD.FLAG_GONAD_OUTLIER==True, 'GONAD_WEIGHT'] = np.nan
df_FD.loc[df_FD.FLAG_GONAD_OUTLIER==True, 'GONAD_WEIGHT']

5790    NaN
13647   NaN
23059   NaN
23295   NaN
24542   NaN
26462   NaN
33026   NaN
Name: GONAD_WEIGHT, dtype: float64

## FLAG_MULTIPLE_SAMPLE_POSSIBILITIES and FLAG_MISNUMBERED_FISH_DETAILS: 

In [19]:
# df_FD.loc[df_FD.FLAG_MULTIPLE_SAMPLE_POSSIBILITIES==True]

In [20]:
# df_FD.loc[df_FD.FLAG_MISNUMBERED_FISH_DETAILS==True]

## FLAG_LEN_WT_RATIO_OUTLIER: 

In [21]:
# df_FD.loc[df_FD.FLAG_LEN_WT_RATIO_OUTLIER==True, ['FL_STD', 'WEIGHT']]

#  ==================================
# LENGTH FREQUENCIES
#  ==================================

## FLAG_SITE: ambiguous site

In [22]:
# TODO: if site is changed away from SITE1, update id to match

# df_LF.loc[df_LF.FLAG_SITE==True]

## FLAG_AM_PM_PERIOD: leave as is

#  ==================================
# SAMPLES
#  ==================================

## FLAG_DATETIME: null datetime

In [23]:
# TODO: if date is changed, update id to match

# df_SD.loc[df_SD.FLAG_DATETIME==True]

## FLAG_HOURS_FISHED: hours_fished = "maximum "

In [24]:
# CONFIRM
df_SD.loc[df_SD.FLAG_HOURS_FISHED==True, 'hours_fished'] = 18
df_SD.loc[df_SD.FLAG_HOURS_FISHED==True, 'hours_fished']

13667    18
13674    18
13682    18
13691    18
13702    18
13713    18
13724    18
13736    18
13749    18
13759    18
13771    18
13782    18
13794    18
13807    18
13820    18
13832    18
13844    18
13857    18
13870    18
13881    18
13893    18
13905    18
13918    18
13931    18
13943    18
13955    18
13966    18
13976    18
Name: hours_fished, dtype: object

## FLAG_SITE: ambiguous site data

In [25]:
# TODO: if site is updated, update id to match

# these are currently importing as 1A
df_SD.loc[df_SD.FLAG_SITE==True, 'SITE_NO']

6019    1A,8
6028    1A,8
6037    1A,8
6046    1A,8
6055    1A,8
6064    1A,8
6073    1A,8
6081    1A,8
6089    1A,8
6097    1A,8
6106    1A,8
6115    1A,8
6124    1A,8
6132    1A,8
6141    1A,8
6150    1A,8
6158    1A,8
6167    1A,8
6183    1A,8
6192    1A,8
6201    1A,8
6209    1A,8
6217    1A,8
Name: SITE_NO, dtype: object

## FLAG_AM_PM_PERIOD_DISCREPANCIES: FD and LF inconsistent
#### SAMPLES, LENGTH FREQUENCIES, and FISH DETAILS all flagged
* The samples will be where the AM PM data will be stored, so they are the only updates needed, outside of ghost samples, which can be updated as information becomes available.

In [26]:
df_SD.loc[df_SD.FLAG_AM_PM_PERIOD_DISCREPANCIES==True]

,DIST,RIVER,NAME,code,GEAR,SITE_NO,no_nets,YEAR,MM,DD,Week,catch_lbs,catch_kg,hours_fished,zone,last_name,comments,bycatch_sbass,bycatch_shad,bycatch_other,DATETIME,SITE1,SITE2,remarks,id,total_fish_preserved,total_fish_measured,AM_PM_PERIOD,wt_lbs,FLAG_DATETIME,FLAG_HOURS_FISHED,FLAG_SITE,FLAG_AM_PM_PERIOD_DISCREPANCIES
5108,2,SWMARG,Daniel Stewart,NaN,81,33,NaN,1993,5,13,3,1100.000,499.000,NaN,upper,STEWART,NaN,NaN,NaN,NaN,1993-05-13,33,<NA>,NaN,1993051333,34.000,210.000,AM,115.000,NaN,NaN,NaN,True
8224,2,SWMARG,Pierre Chiasson,NaN,81,26,1,2001,6,15,7,500.000,226.800,NaN,lower,Chiasson,NaN,NaN,NaN,NaN,2001-06-15,26,<NA>,NaN,2001061526,51.000,283.000,AM,136.000,NaN,NaN,NaN,True
10432,2,SWMARG,Pierre Chiasson,NaN,81,26,1,2007,6,2,5,9600.000,4354.500,8,lower,Chiasson,NaN,NaN,NaN,NaN,2007-06-02,26,<NA>,NaN,2007060226,32.000,290.000,AM,130.000,NaN,NaN,NaN,True
13498,2,SWMARG,Donelda M Gillis,NaN,81,47,1,2014,6,10,7,450.000,204.100,7,upper,Gillis,NaN,NaN,NaN,NaN,2014-06-10,47,<NA>,NaN,2014061047,38.000,246.000,AM,89.000,NaN,NaN,NaN,True


In [27]:
# df_FD.loc[df_FD.FLAG_AM_PM_PERIOD_DISCREPANCIES==True]

In [28]:
# df_LF.loc[df_LF.FLAG_AM_PM_PERIOD_DISCREPANCIES==True]

## FLAG_NO_MATCHING_SAMPLE: no SAMPLE matching LF and/or FD
* these will end up being matched with ghost samples (below)

In [29]:
# df_FD.loc[df_FD.FLAG_NO_MATCHING_SAMPLE==True]

In [30]:
# df_LF.loc[df_LF.FLAG_NO_MATCHING_SAMPLE==True]

## MAKE GHOST SAMPLES
### for unmatched fish details and length frequencies
### NOTE:
Some ambiguous length frequencies and fish details match with eachother. However, we do not know which sample and length frequency are from the same sample. Therefore, we should make sure none of the ambiguous entries ever match automatically, and they are matched manually (eventually) in dm_apps, once that feature is implemented.

Although there are no ambiguous samples that match with length frequencies or fish details, just for throroughness, we should revise id to avoid this possibility as well. 

Therefore, we will arbitrarily add 20 to months in ids of ambiguous fish details, and 40 to months in ids of ambiguous length frequencies. These numbers have no possibility to match a false positive, and they still uniquely identify their date and site.

In [31]:
# to disambiguate ids, add 20 to all months for FD with id>2024000000, add 40 to all months for LF with id>2024000000

# these were matched using period, and corroborated using site notes and visual inspection
matched_period = [
    2998050105, 3000060726, 3000060926, 3001051605, 3009052026, 3009052226, 3010051126, 3010051926, 3010052726, 
    4000060726, 4000060926, 4001051605, 4009052026, 4009052226, 4010051126, 4010051926, 4010052726
]

df_FD.loc[(df_FD.id>2024000000) & (~df_FD.id.isin(matched_period)), 'id'] += 200000
df_LF.loc[(df_LF.id>2024000000) & (~df_LF.id.isin(matched_period)), 'id'] += 400000

# dubious matches - CONFIRM
disambiguate_dubious_id_matches = False  ### CONFIRM ###
# in general these matches look good, but they have matches like 1 ~ 1,8 ~ 1,8 (not exact matches, but the first entry always matches)
# either we can imported matched and include notes, or disambiguate and match manually later
df_ambiguous_match_notes = pd.read_pickle('df_ambiguous_match_notes.pickle')
if disambiguate_dubious_id_matches:
    df_FD.loc[df_FD.id.isin(df_ambiguous_match_notes.id), 'id'] += 200000
    df_LF.loc[df_LF.id.isin(df_ambiguous_match_notes.id), 'id'] += 400000

# verified samples are all sample ids before creating ghost samples
verified_samples = set(df_SD.id)

# temporarily add remarks to df_LF in order to note site
df_LF['remarks'] = 'Site: ' + df_LF['site'].fillna('None')

# JOIN FD and LF without an SD match, and combine remarks, date, and site
temp = pd.merge(
    df_LF[~df_LF.id.isin(df_SD.id)].drop_duplicates('id')[['id', 'DATETIME', 'SITE1', 'remarks']], 
    df_FD[~df_FD.id.isin(df_SD.id)].drop_duplicates('id')[['id', 'DATETIME', 'SITE1', 'remarks']],
    on='id',
    how='outer'
).reset_index(drop=True)
temp.loc[temp.remarks_x.isnull() | temp.remarks_x.isnull(), 'remarks'] = temp['remarks_x'].fillna('') + temp['remarks_y'].fillna('')
temp.loc[temp.remarks_x.notnull() & temp.remarks_x.notnull(), 'remarks'] = temp['remarks_x'].fillna('') + '; ' + temp['remarks_y'].fillna('') # semicolon between if both exist
temp['DATETIME'] = temp['DATETIME_x'].fillna(temp['DATETIME_y'])
temp['SITE1'] = temp['SITE1_x'].fillna(temp['SITE1_y'])
temp = temp[['id', 'DATETIME', 'SITE1', 'remarks']]

# concatenate to the bottom of the df_SD dataframe
df_SD = pd.concat([
    df_SD, 
    temp
]).reset_index(drop=True)

# remove remarks from df_LF, not needed for df_LF
df_LF.drop('remarks', axis=1)

# add a Flag to indicate ghost samples for import
df_SD.loc[~df_SD.id.isin(verified_samples), 'FLAG_GHOST_SAMPLE'] = True

# add remarks about ghost samples
df_SD.loc[~df_SD.id.isin(verified_samples), 'remarks'] += '; GHOST SAMPLE: created to match with unmatched Fish Details and/or Length Frequencies'

# either way, add  notes to samples with dubious matches
for i, row in df_SD[df_SD.id.isin(df_ambiguous_match_notes.id)].iterrows():
    current_id = row['id']
    if df_SD.loc[df_SD.id==current_id, 'remarks'].isnull().any():
        df_SD.loc[df_SD.id==current_id, 'remarks'] = df_ambiguous_match_notes.loc[df_ambiguous_match_notes.id==current_id, 'SITE_AMBIGUITIES'].values[0]
    else:
        df_SD.loc[df_SD.id==current_id, 'remarks'] += '; ' + df_ambiguous_match_notes.loc[df_ambiguous_match_notes.id==current_id, 'SITE_AMBIGUITIES'].values[0]
    df_SD.loc[df_SD.id==current_id, 'FLAG_AMBIGUOUS_MATCH'] = True

## REMERGE JOINED DATA
(previously merged without ghost samples, remerge with ghost data)

In [32]:
# drop previously merged columns (to be re-merged)
df_SD = df_SD.drop(['total_fish_preserved', 'total_fish_measured', 'AM_PM_PERIOD', 'wt_lbs'], axis=1)

# JOIN with Fish Details table to get total_fish_preserved
# NOTE: this is an estimate, assuming all fish details are accounted for. This is the best information available
df_SD = pd.merge(
    df_SD, 
    pd.merge(df_SD, df_FD, on='id').groupby('id').count()['FISH_NO'].reset_index(),
    on='id',
    how='left'
).rename({'FISH_NO':'total_fish_preserved'}, axis=1)

# JOIN with Length Frequencies table to get total_fish_measured
# NOTE: this is an estimate, assuming all length frequencies are accounted for. This is the best information available
df_SD = pd.merge(
    df_SD, 
    pd.merge(df_SD, df_LF, on='id').groupby('id').sum()['freq'].reset_index(),
    on='id',
    how='left'
).rename({'freq':'total_fish_measured'}, axis=1)

# # JOIN with Fish Details and Length Frequencies to get AM_PM_PERIOD 
# # NOTE: discrepancies flagged between Length Frequencies and Fish Details
df_period = pd.merge(
    df_FD[df_FD.PERIOD.notnull()].groupby('id').first().reset_index()[['id', 'PERIOD']],
    df_LF[df_LF.period.notnull()].groupby('id').first().reset_index()[['id', 'period']], 
    on='id',
    how='outer'
)
df_period['AM_PM_PERIOD'] = df_period['PERIOD'].fillna(df_period['period'])

df_SD = pd.merge(
    df_SD, 
    df_period[['id', 'AM_PM_PERIOD']],
    on='id',
    how='left'
)

# JOIN with Length Frequency table to get sample weight
df_SD = pd.merge(
    df_SD, 
    df_LF[['id', 'wt_lbs']],
    on='id', 
    how='left'  # all samples 
).drop_duplicates().reset_index(drop=True)

# Fix Datetime Format

In [33]:
# some times merged into DATETIME from other df via ghost samples () 
# reduce to date and AM / PM period
df_SD['DATETIME'] = pd.to_datetime(df_SD['DATETIME'].dt.date)

#  ==================================
# LF Grouped
#  ==================================

## Recreate df_LF_grouped with new ambiguous ids for LF entries (+40 to month if ambiguous)

In [34]:
# group by length bins, include only imported columns 
df_LF_grouped = df_LF.groupby(['id', 'length_bin_id']).sum('freq').reset_index()[['freq', 'length_bin_id', 'id']].rename({
    'freq': 'count', 
    'length_bin_id': 'length_bin_id', 
    'id': 'sample_id'
}, axis=1).reset_index(drop=True)[['sample_id', 'length_bin_id', 'count']]

In [35]:
# ambiguous entries, month bit (+40 to disambiguate)
min([int(str(x)[4:6]) for x in list(df_LF_grouped[df_LF_grouped.sample_id>2024000000].sample_id.unique())])

5

In [36]:
# regular entries, month bit
max([int(str(x)[4:6]) for x in list(df_LF_grouped[df_LF_grouped.sample_id<2024000000].sample_id.unique())])

6

#  ==================================
# COMMENTS
#  ==================================

CONVERT E.MACFARLANE, Eric Mac, Eric MacFarlane, J McFarlane to John Eric MacFarlane.

JA Coady keep as is

Multiple sites, keep as is. 

I see 118 samples from 1989 that are associated with ‘blank’. Can you convert ‘blank’ to Unknown and upload.

#### based on the comment: 93 and 92 are both 'John Eric MacFarlane' - should be added to sites table

In [37]:
# append these and update the import script
df_Site = pd.concat([
    df_Site,
    pd.DataFrame([['John Eric MacFarlane'], ['JA Coady']], columns=['site'])
], axis=0).reset_index(drop=True)

##### MORE COMMENTS

#  ==================================
# SAVE DATA
#  ==================================

In [38]:
# reorder columns for clarity - all flags at the end
df_SD = df_SD[[x for x in list(df_SD.columns) if 'FLAG' not in str(x)] + [x for x in list(df_SD.columns) if 'FLAG' in str(x)]]
df_FD = df_FD[[x for x in list(df_FD.columns) if 'FLAG' not in str(x)] + [x for x in list(df_FD.columns) if 'FLAG' in str(x)]]
df_LF = df_LF[[x for x in list(df_LF.columns) if 'FLAG' not in str(x)] + [x for x in list(df_LF.columns) if 'FLAG' in str(x)]]

In [39]:
# save pickle files of dataframes
if save_pickles := False:
    df_SD.to_pickle('df_SD_cleaned.pickle')
    df_FD.to_pickle('df_FD_cleaned.pickle')
    df_LF.to_pickle('df_LF_cleaned.pickle')
    df_LF_grouped.to_pickle('df_LF_grouped_cleaned.pickle')
    df_Site.to_pickle('df_Site_cleaned.pickle')
    df_TrapSupervisors.to_pickle('df_TrapSupervisors_cleaned.pickle')

# save files to this workbook drive
if save_csv := False:
    df_SD.to_csv('gaspereau_sample_data_cleaned.csv', index=False)
    df_FD.to_csv('gaspereau_fish_details_cleaned.csv', index=False)
    df_LF.to_csv('gaspereau_length_frequencies_cleaned.csv', index=False)
    df_Site.to_csv('gaspereau_sites_cleaned.csv', index=False)
    df_TrapSupervisors.to_csv('gaspereau_trap_supervisors_cleaned.csv', index=False)
    df_LF_grouped.to_csv('gaspereau_LF_grouped_cleaned.csv', index=False)
    
# upload to temp folder for import into Kevin's local dm_apps
import_file_location = 'C:\\Users\\CARRK\\Documents\\Repositories\\dm_app_root\\dm_apps\\herring\\temp\\'

if upload_csv_to_temp_folder := False:
    df_SD.to_csv(import_file_location + 'gaspereau_sample_data.csv', index=False)
    df_FD.to_csv(import_file_location + 'gaspereau_fish_details.csv', index=False)
    df_Site.to_csv(import_file_location + 'gaspereau_sites.csv', index=False)
    df_TrapSupervisors.to_csv(import_file_location + 'gaspereau_trap_supervisors.csv', index=False)
    df_LF_grouped.to_csv(import_file_location + 'gaspereau_LF_grouped.csv', index=False)

# OLD STUFF - recheck later

# Check Import Issues

In [40]:
checkthese = [
    1993052010, 1993053010, 2016053100, 2016061400, 2016062100, 2017053092, 2017060692, 2018052900, 2018060800, 
    2018062700, 2019060794, 2019060793, 2019061493, 2019062594, 1989060100, 1989060200, 1989060700, 2011060414, 
    2016053195, 2016061495, 2016062195, 2018052992, 2018062792, 2018060892, 2021052092, 2021060192, 2021060392, 
    2021061592, 2021062392
]

# these were rejected then recreated
len(list(df_SD[df_SD.id.isin(checkthese)].id)), len(checkthese)

(29, 29)

In [41]:
# why were they rejected
df_SD[df_SD.id.isin(checkthese)].SITE1.unique()

array(['10', <NA>, 92, 94, 93, 14, 95], dtype=object)

In [42]:
# none of the sites match
df_Site.site.unique()

array([1, '1A', '1B', 2, 4, 5, 6, 7, 8, 9, 11, 12, 15, 17, 18, 19, 20, 21,
       23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40,
       41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 59,
       60, 61, 62, 63, 64, 65, 66, 67, 68, 'John Eric MacFarlane',
       'JA Coady'], dtype=object)

In [43]:
# SOLUTION: if a site is unmatched, make it None, don't reject it
# didn't happen, but similar for trap_supervisor not matching FK table

# After Import Checks

In [44]:
rejected_FD = list(df_FD[df_FD.FLAG_MISNUMBERED_FISH_DETAILS==True].id.unique())
rejected_SD = list(df_SD[df_SD.FLAG_DATETIME==True].id.unique())
rejected_id = rejected_FD + rejected_SD

In [45]:
# -> CORRECT
# how many total samples were imported?
df_SD[df_SD.FLAG_DATETIME.isnull()].shape[0]

15467

In [46]:
# what about LF, FD, both?
(
    df_SD[(df_SD.FLAG_DATETIME.isnull()) & (df_SD.id.isin(df_LF.id.unique()))].shape[0],
    df_SD[(df_SD.FLAG_DATETIME.isnull()) & (df_SD.id.isin(df_FD.id.unique()))].shape[0],
    df_SD[(df_SD.FLAG_DATETIME.isnull()) & (df_SD.id.isin(df_LF.id.unique())) & (df_SD.id.isin(df_FD.id.unique()))].shape[0]
)
# -> 916, 1149, 803
# yes, meh, yes

(916, 1156, 821)

In [47]:
# -> CORRECT
# why are we missing 7 FD? were they rejected?
# -> YES, exactly 7 were rejected (invalid kwargs)

In [48]:
# without any FD or LF?
df_SD[~(df_SD.id.isin(df_LF.id.unique())) & ~(df_SD.id.isin(df_FD.id.unique())) & (df_SD.FLAG_DATETIME.isnull())].shape[0]
# -> 14223
# close, but how are there more in dm_apps?

14216

In [49]:
# maybe the rejected FD were still imported
rejected_FD = [2993252737, 2993252952, 2993260933, 2998250105, 3010252925, 3010250541, 3010261141] # all of these samples were imported as ghost samples
df_SD[~(df_SD.id.isin(df_LF.id.unique())) & (~(df_SD.id.isin(df_FD.id.unique())) | df_SD.id.isin(rejected_FD)) & (df_SD.FLAG_DATETIME.isnull())].shape[0]
# -> CORRECT (yuck code, but it matches)

14222

In [50]:
# -> CORRECT
# how many ambiguous samples?  6
# do these match with FD or LF?  No. (correct)
df_SD[~(df_SD.FLAG_GHOST_SAMPLE == True) & (df_SD.id > 2024000000) & (df_SD.FLAG_DATETIME.isnull())][['DATETIME']+list(df_SD.columns)]

,DATETIME,DIST,RIVER,NAME,code,GEAR,SITE_NO,no_nets,YEAR,MM,DD,Week,catch_lbs,catch_kg,hours_fished,zone,last_name,comments,bycatch_sbass,bycatch_shad,bycatch_other,DATETIME,SITE1,SITE2,remarks,id,total_fish_preserved,total_fish_measured,AM_PM_PERIOD,wt_lbs,FLAG_DATETIME,FLAG_HOURS_FISHED,FLAG_SITE,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_GHOST_SAMPLE,FLAG_AMBIGUOUS_MATCH
2683,1988-05-23,2.000,SWMARG,Michael D Gillis,NaN,81.000,48,NaN,1988,5,23,<NA>,7015.000,3182.000,13,upper,Gillis,NaN,NaN,NaN,NaN,1988-05-23,48,<NA>,NaN,2988052348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2684,1988-05-23,2.000,SWMARG,Michael D Gillis,NaN,81.000,48,NaN,1988,5,23,<NA>,4008.000,1818.000,13,upper,Gillis,NaN,NaN,NaN,NaN,1988-05-23,48,<NA>,NaN,3988052348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6558,1997-06-12,2.000,SWMARG,Michael D Gillis,NaN,81.000,58,1,1997,6,12,7,900.000,408.200,NaN,upper,Gillis,NaN,NaN,NaN,NaN,1997-06-12,58,<NA>,NaN,2997061258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6559,1997-06-12,2.000,SWMARG,Michael D Gillis,NaN,81.000,58,1,1997,6,12,7,200.000,90.700,NaN,upper,Gillis,NaN,NaN,NaN,NaN,1997-06-12,58,<NA>,NaN,3997061258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9276,2004-06-10,2.000,SWMARG,Charles McDaniel,NaN,81.000,1,1,2004,6,10,7,1150.000,521.600,15,lower,McDaniel,NaN,NaN,NaN,NaN,2004-06-10,1,<NA>,NaN,3004061001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9277,2004-06-10,2.000,SWMARG,Charles McDaniel,NaN,81.000,1,1,2004,6,10,7,1125.000,510.300,6.5,lower,McDaniel,NaN,NaN,NaN,NaN,2004-06-10,1,<NA>,NaN,4004061001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# -> CORRECT
# how many ghost sites? with LF? FD? both?
(
    df_SD[df_SD.FLAG_GHOST_SAMPLE == True].shape[0],
    df_SD[(df_SD.FLAG_GHOST_SAMPLE == True) & (df_SD.id.isin(df_LF.id.unique()))].shape[0],
    df_SD[(df_SD.FLAG_GHOST_SAMPLE == True) & (df_SD.id.isin(df_FD.id.unique())) & ~(df_SD.id.isin(rejected_id))].shape[0],
    df_SD[(df_SD.FLAG_GHOST_SAMPLE == True) & (df_SD.id.isin(df_LF.id.unique())) & (df_SD.id.isin(df_FD.id.unique())) & ~(df_SD.id.isin(rejected_id))].shape[0]
)
# -> 303, 185, 241, 130

(285, 184, 241, 146)